T5

In [ ]:
!pip install -U python-terrier
!pip install -U git+https://github.com/terrierteam/pyterrier_doc2query.git

     |████████████████████████████████| 95 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 28.8 MB/s 
     |████████████████████████████████| 69 kB 6.4 MB/s 
     |████████████████████████████████| 222 kB 44.4 MB/s 
     |████████████████████████████████| 1.8 MB 23.3 MB/s 
     |████████████████████████████████| 636 kB 52.6 MB/s 
     |████████████████████████████████| 294 kB 51.1 MB/s 
     |████████████████████████████████| 72 kB 1.8 MB/s 
     |████████████████████████████████| 6.3 MB 39.9 MB/s 
     |████████████████████████████████| 126 kB 73.6 MB/s 
     |████████████████████████████████| 291 kB 70.5 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.0-py3-none-any.whl size=102213 sha256=4d7f473cdda97d77ca38ad1caac3d63e741161f42e764765a095414bcfb57699
  Stored in directory: /root/.cache/pip/wheels/e3/84/1e/68c08f14e2481e2b3e7c1a2c24bb1220712bc3f5d3896c28df
  Created wheel for ir-measures: filen

In [ ]:
import pyterrier as pt
import os
import pandas as pd
import pyterrier_doc2query
import sys

if not pt.started():
    pt.init()

terrier-assemblies 5.6  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [ ]:
dataset_name = 'irds:vaswani'
path_index_vaswani = '/content/index_vaswani'
props_path = '/data.properties'
dataset = pt.datasets.get_dataset(dataset_name)

if not os.path.exists(path_index_vaswani):
    indexer = pt.index.IterDictIndexer(path_index_vaswani)
    index_ref = indexer.index(dataset.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref = pt.IndexRef.of(path_index_vaswani + props_path)

index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics()
qrels = dataset.get_qrels()
evals = ['map', 'ndcg_cut.10', 'P.10', 'mrt']
br = pt.BatchRetrieve(index, wmodel='BM25')

[INFO] [starting] building docstore
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:06, 319kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 10.0%| 213k/2.13M [00:00<00:02, 649kB/s]

[INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [3.57MB/s]
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [3.46MB/s]
docs_iter: 100%|███████████████████████| 11429/11429 [00:01<00:00, 9546.86doc/s]
[INFO] [finished] docs_iter: [00:01] [11429doc] [9536.32doc/s]
[INFO] [finished] building docstore [1.21s]


vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [ ]:
def experiment_br(retrs, labels):
  return pt.Experiment(retrs, topics, qrels, names=labels, eval_metrics=evals)

In [ ]:
if not os.path.exists('t5-base.zip'):
  !wget https://git.uwaterloo.ca/jimmylin/doc2query-data/raw/master/T5-passage/t5-base.zip
  !unzip t5-base.zip

path_index_t5 = '/content/index_t5'
doc2query = pyterrier_doc2query.Doc2Query(out_attr='text', batch_size=8)
indexer_t5 = doc2query >> pt.IterDictIndexer(path_index_t5)
index_ref_t5 = indexer_t5.index(dataset.get_corpus_iter())
br_doct5 = pt.BatchRetrieve(index_ref_t5, wmodel='BM25')
# br_doct5 = br >> pt.get_text(dataset) >> doc2query >> pt.text.scorer(body_attr='querygen', wmodel='BM25')

expr_doct5 = experiment_br([br, br_doct5], ['BM25', 'BM25 >> T5'])
print(pd.DataFrame(expr_doct5))

Doc2query using cpu


vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

         name       map  ndcg_cut.10      P.10        mrt
0        BM25  0.296517     0.446609  0.352688  84.820111
1  BM25 >> T5  0.171971     0.325610  0.248387  28.236750


end of fun.